In [2]:
# gmailer init


In [21]:
import datetime as dt

import smtplib
import mimetypes
from email.message import EmailMessage



# Email class

In [44]:
sender = "jaime.meriz13@gmail.com"
recipients = ["hexarunner@gmail.com"]
subject = "DEV" + "-" "Jmail" + "-" + str(dt.date.today())
body = "body\nyou'rs truly\nDr.Lopez"
password = "password"

In [47]:

def send_mail(sender, recipients, subject, body, password):

    email = EmailMessage()

    email["Sender"] = sender
    email["Recipients"] = recipients
    email["Subject"] = subject
    email.body = body

    return email



In [48]:
send_mail(sender, recipients, subject, body, password)

AttributeError: 'list' object has no attribute 'startswith'